<a href="https://colab.research.google.com/github/dishambha/NyayaGPT/blob/main/Agentic_RAG_Application_for_LAW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-openai langchain-community pypdf faiss-cpu sentence-transformers

In [ ]:
!pip install langchain-groq


In [27]:
!pip install sentence-transformers

# Part 1: Importing Necessary Libraries
# ----------------------------------------------------------------


In [39]:
# ==============================================================================
# Corrected Part 1: Importing All Necessary Libraries
# ==============================================================================

import os
from google.colab import userdata # For securely handling API keys in Colab

# For loading and preparing your documents
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# For the vector database and the free embedding model
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# For the Groq LLM which will be the agent's "brain"
from langchain_groq import ChatGroq

# For building the agent and its tools
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

# For managing the conversation's memory
from langchain_core.messages import HumanMessage, AIMessage

# Part 2 (Updated for Groq): API Key and Configuration

In [29]:

# ----------------------------------------------------

# This line now retrieves the API key named 'GROQ_API_KEY' which you must create in Colab's Secrets Manager (🔑).
# It's a secure way to handle keys without hard-coding them.
# Make sure to enable 'Notebook access' for the secret.
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

# Part 3: The Reusable "Factory" Function for Tool Creation

In [30]:
# Add this new import statement at the top of your script
from langchain_community.embeddings import HuggingFaceEmbeddings

# This is the corrected function
def create_pdf_retriever_tool(pdf_path, tool_name, tool_description):
    """This function now uses a free Hugging Face model for embeddings."""

    # 1. Loading the document
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # 2. Chunking the document
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)

    # 3. Creating Embeddings (THE CORRECTED PART)
    # We replace the OpenAI line with this one to use a free model.
    # The first time this runs, it will download the model (approx. 220MB).
    embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    # The rest of the function works perfectly with the new model
    vector_store = FAISS.from_documents(docs, embeddings_model)

    # 4. Creating the Retriever
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})

    # 5. Wrapping the retriever in a Tool
    retriever_tool = create_retriever_tool(retriever, name=tool_name, description=tool_description)

    print(f"Tool '{tool_name}' for '{pdf_path}' created successfully with free embeddings.")

    return retriever_tool

# Part 4: Building Your Specific Tools

In [32]:

# ------------------------------------

# Create the first tool for the BNS document.
# We provide the file path, a simple name for the tool, and a detailed description of its purpose.
bns_tool = create_pdf_retriever_tool(
    pdf_path="BNS.pdf", # Make sure this path matches your uploaded file name in Colab
    tool_name="bns_law_search",
    tool_description="Use this tool to search for the definitions of crimes, punishments, and substantive criminal law as defined in the Bharatiya Nyaya Sanhita (BNS)."
)

# Create the second tool for the BNSS document.
# Notice the description is different, telling the agent this tool is for procedural law.
bnss_tool = create_pdf_retriever_tool(
    pdf_path="BNSS.pdf", # Make sure this path matches your uploaded file name in Colab
    tool_name="bnss_procedure_search",
    tool_description="Use this tool to search for legal procedures, rules for investigation, arrest, bail, and trial processes as defined in the Bharatiya Nagarik Suraksha Sanhita (BNSS)."
)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Tool 'bns_law_search' for 'BNS.pdf' created successfully with free embeddings.
Tool 'bnss_procedure_search' for 'BNSS.pdf' created successfully with free embeddings.


# Part 5 (Updated for Groq): Creating the Agent and Executor

In [38]:
# Part 5 (Updated for Groq with a ReAct Agent): Creating the Agent and Executor
# -----------------------------------------------------------------------------

# --- Import the correct agent creation function ---
# We are switching from 'create_openai_functions_agent' to the more general 'create_react_agent'
from langchain.agents import create_react_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain import hub
import os
from langchain_core.messages import HumanMessage, AIMessage

# Your API key setup and tools list remain the same
# os.environ["GROQ_API_KEY"] = ...
tools = [bns_tool, bnss_tool]

# --- Use a more compatible prompt template ---
# We pull a prompt designed for the ReAct framework, which is more universal.
prompt = hub.pull("hwchase17/react-chat")

# Initialize the Groq LLM as before
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0)

# --- Create the agent using the new function ---
# This assembles the agent using the ReAct logic.
agent = create_react_agent(llm, tools, prompt)

# The AgentExecutor remains the same. It's the runtime for your agent.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Now, run your query again!
# This setup is much more likely to make the Llama 3 model show its reasoning and use your tools.
response = agent_executor.invoke({
    "input": "What is the punishment for assault under the BNS, and what is the procedure for filing a report according to the BNSS?",
    "chat_history": [] # Add this line to provide an empty chat history
})

print("\n--- FINAL SYNTHESIZED ANSWER ---")
print(response["output"])

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: bns_law_search
Action Input: What is the punishment for assault under the BNSExplanation.—A threat to injure the reputation of any deceased person in whom the
person threatened is interested, is within this section.
Illustration.
A, for the purpose of inducing B to resist from prosecuting a civil suit, threatens to
burn B’s house. Ais guilty of criminal intimidation.
(2) Whoever commits the offence of criminal intimidation shall be punished with
imprisonment of either description for a term which may extend to two years, or with fine, or
with both.
(3) Whoever commits the offence of criminal intimidation by threatening to cause
death or grievous hurt, or to cause the destruction of any property by fire, or to cause an
offence punishable with death or imprisonment for life, or with imprisonment for a term which
may extend to seven years, or to impute unchastity to a woman, shall be punished with
impris

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Do I need to use a tool? Yes
Action: bnss_procedure_search
Action Input: What is the procedure for filing a report according to the BNSS?any, which are to be produced in evidence;
Supply to
accused of
copy of police
report and
other
documents.
Supply of
copies of
statements
and documents
to accused in
other cases
triable by
Court of
Session.
Commitment
of case to Court
of Session when
offence is
triable
exclusively by
it.
Sec. 1]
THE GAZETTE OF INDIA EXTRAORDINARY
75
___________________________________________________________
__________________________________________________________
__________________________________________________________
__________________________________________________________
__________________________________________________________
__________________________________________________________

of Session.
Language of
record of
evidence.
Procedure in
regard to such
evidence when
completed.
Sec. 1]
THE GAZETTE OF INDIA EXTRAORDINARY
95
_____________________________

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Thought: Do I need to use a tool? No
Final Answer: According to the BNS, the punishment for assault is imprisonment of either description for a term which may extend to two years, or with fine, or with both (Section 134). Additionally, if the assault is committed with intent to dishonour the person, the punishment is imprisonment of either description for a term which may extend to two years, or with fine, or with both (Section 133).

As for the procedure for filing a report according to the BNSS, the relevant sections provide guidelines for the supply of copies of police reports and other documents to the accused, commitment of cases to the Court of Session, language of record of evidence, and procedure in regard to such evidence when completed. However, the specific procedure for filing a report is not explicitly stated in the provided text. It is recommended to consult the relevant sections of the BNSS for more information on the procedure for filing a report.

> Finished chain.

--

# Part 6: Running the Agent to Get a Final Output

In [37]:

# ==============================================================================
# Part 6 (Corrected): Running the Agent to Get a Final Output
# ==============================================================================

# Import message types needed to manage the conversation history
from langchain_core.messages import HumanMessage, AIMessage

# Create an empty list to hold the conversation history. This is the main fix.
chat_history = []

# Define your complex query
query = "What is the punishment for assault under the BNS, and what is the procedure for filing a report according to the BNSS?"

# Invoke the agent, now passing the 'input' AND the 'chat_history'
# This dictionary now contains all the keys the prompt template expects.
response = agent_executor.invoke({
    "input": query,
    "chat_history": chat_history
})

# Print the final answer from the agent
print("\n--- FINAL SYNTHESIZED ANSWER ---")
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: bns_law_search
Action Input: What is the punishment for assault under the BNS?of private defence.
Explanation 2.—Whether the provocation was grave and sudden enough to mitigate
the offence, is a question of fact.
132.Whoever assaults or uses criminal force to any person being a public servant in
the execution of his duty as such public servant, or with intent to prevent or deter that
person from discharging his duty as such public servant, or in consequence of anything
done or attempted to be done by such person in the lawful discharge of his duty as such
public servant, shall be punished with imprisonment of either description for a term which
may extend to two years, or with fine, or with both.
133.Whoever assaults or uses criminal force to any person, intending thereby to
dishonour that person, otherwise than on grave and sudden provocation given by that
person, shall be punished with imprisonment 

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Do I need to use a tool? Yes
Action: bnss_procedure_search
Action Input: What is the procedure for filing a report according to the BNSS?any, which are to be produced in evidence;
Supply to
accused of
copy of police
report and
other
documents.
Supply of
copies of
statements
and documents
to accused in
other cases
triable by
Court of
Session.
Commitment
of case to Court
of Session when
offence is
triable
exclusively by
it.
Sec. 1]
THE GAZETTE OF INDIA EXTRAORDINARY
75
___________________________________________________________
__________________________________________________________
__________________________________________________________
__________________________________________________________
__________________________________________________________
__________________________________________________________

of Session.
Language of
record of
evidence.
Procedure in
regard to such
evidence when
completed.
Sec. 1]
THE GAZETTE OF INDIA EXTRAORDINARY
95
_____________________________

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Thought: Do I need to use a tool? No
Final Answer: According to the BNS, the punishment for assault can vary depending on the circumstances. For example, assaulting a public servant in the execution of their duty can lead to imprisonment of up to two years, or a fine, or both (Section 132). Assaulting someone with the intent to dishonor them can also lead to imprisonment of up to two years, or a fine, or both (Section 133). 

As for the procedure for filing a report according to the BNSS, it appears that the relevant sections are related to the supply of copies of police reports and other documents to the accused, commitment of cases to the Court of Session, and the language of record of evidence. However, the exact procedure is not clearly outlined in the provided text. It may be necessary to consult the full text of the BNSS or seek further clarification.

> Finished chain.

--- FINAL SYNTHESIZED ANSWER ---
According to the BNS, the punishment for assault can vary depending on the ci

In [40]:
# After your first response...
# 1. Update the history with the last interaction
chat_history.append(HumanMessage(content=query))
chat_history.append(AIMessage(content=response["output"]))

# 2. Ask a follow-up question
follow_up_query = "Tell me more about the 'commitment of cases to the Court of Session' that you mentioned."

# 3. Invoke the agent again with the UPDATED history
response_2 = agent_executor.invoke({
    "input": follow_up_query,
    "chat_history": chat_history
})

print(response_2["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: bnss_procedure_search
Action Input: commitment of cases to the Court of Sessionis of opinion that it will tend to the general convenience of the parties and witnesses to hold
its sittings at any other place in the sessions division, it may, with the consent of the
prosecution and the accused, sit at that place for the disposal of the case or the examination
of any witness or witnesses therein.
(7) The Sessions Judge may, from time to time, make orders consistent with this
Sanhita, as to the distribution of business among such Additional Sessions Judges.
(8) The Sessions Judge may also make provision for the disposal of any urgent
application, in the event of his absence or inability to act, by an Additional Sessions Judge
or if there be no Additional Sessions Judge, by the Chief Judicial Magistrate, and such
Judge or Magistrate shall be deemed to have jurisdiction to deal with any such application.
Ex

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Do I need to use a tool? No
Final Answer: According to the BNSS, the commitment of cases to the Court of Session involves the Sessions Judge making orders for the distribution of business among Additional Sessions Judges, and provision for the disposal of urgent applications in the event of the Sessions Judge's absence or inability to act. The Court of Session shall ordinarily hold its sittings at a specified place, but may sit at another place in the sessions division with the consent of the prosecution and the accused. Additionally, the Magistrate shall notify the Public Prosecutor of the commitment of the case to the Court of Session, and any application filed by the accused or victim shall be forwarded to the Court of Session with the committal of the case.

> Finished chain.
According to the BNSS, the commitment of cases to the Court of Session involves the Sessions Judge making orders for the distribution of business among Additional Sessions Judges, and provision for the disposa